## Dacon  14회 KB 금융문자 분석 모델링 경진대회
## 스미스 요원(팀명)
## 2020년 1월 12일 (제출날짜)

# 모델링 코드 작성방법

A 코드 관련

1) 입상자는 코드 제출 필수. 제출 코드는 예측 결과를 리더보드 점수로 복원할 수 있어야 함

2) 코드 제출시 확장자가 R user는 R or .rmd. Python user는 .py or .ipynb

3) 코드에 ‘/data’ 데이터 입/출력 경로 포함 제출 or R의 경우 setwd(" "), python의 경우 os.chdir을 활용하여 경로 통일

4) 전체 프로세스를 일목요연하게 정리하여 주석을 포함하여 하나의 파일로 제출

5) 모든 코드는 오류 없이 실행되어야 함(라이브러리 로딩 코드 포함되어야 함).

6) 코드와 주석의 인코딩은 모두 UTF-8을 사용하여야 함

 
B 외부 데이터 관련

1) 외부 공공 데이터 (날씨 정보 등) 사용이 가능하나, 코드 제출 시 함께 제출

2) 공공 데이터 외의 외부 데이터는 법적인 제약이 없는 경우에만 사용 가능

3) 외부 데이터를 크롤링할 경우, 크롤링 코드도 함께 제출

## 1. 라이브러리 및 데이터
## Library & Data

In [ ]:
#필요한 Library 설치
!pip install konlpy
!pip install wordcloud
!pip install selenium
!pip install pyperclip

In [7]:
#시드 설정용 라이브러리 import 및 시드 설정
import numpy as np
np.random.seed(7)
import random
random.seed(0)
import tensorflow as tf
if tf.__version__ == '1.15.0' : #1.15.0 버전일 경우 compat.v2 모듈 사용
    tf.compat.v2.random.set_seed(8)
else : # 1.15.0 버전이 아닐 경우 
    tf.random.set_seed(8)

In [8]:
import pandas as pd # 데이터 분석
import json
from tqdm import tqdm
import os
# os.chdir('파일경로')

#전처리용 모델 import
from konlpy.tag import Komoran
from selenium import webdriver
import pyperclip

In [9]:
# 머신러닝 모델 import
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN
from tensorflow.keras.models import Sequential, load_model

In [10]:
#사용한 데이터 불러오기
train = pd.read_csv("../0_DATA/train.csv") # 대회 데이터

## 2. 데이터 전처리
## Data Cleansing & Pre-Processing

In [11]:
text_list = train['text'].values.tolist()

In [ ]:
# train 데이터 전체에 대해 komoran 형태소 분석기 수행

new_text = []
kom = Komoran(userdic='../user_pos_dic.txt') #사용자 사전 사용시 괄호 안에 userdic='경로/user_pos_dic.txt'
for i, document in tqdm(enumerate(text_list)):
    pos = kom.pos(document)
    clean_words = []
    for p in pos:
        if p[1] == 'NNP' or p[1] == 'NNG' or p[1] == 'SL':
            clean_words.append(p[0])
    document = ' '.join(clean_words)
    new_text.append(document)

58131it [04:30, 299.70it/s]

In [7]:
#형태소분석 결과를 train 데이터프레임에 추가
train['pos'] = pd.Series(new_text)

## Crawling Code(크롤링 진행 시 기입)

In [21]:
#Komoran 형태소 분석기 수행 후, 제대로 형태소분석이 되지 않은 문자들 따로 분류
train_non_miss = train[train['pos']!='']
train_miss = train[train['pos']=='']

In [ ]:
#형태소 분석이 되지 않은 문자의 개수
len(train_miss)

In [22]:
# Komoran의 형태소분석이 진행되지 않은 문장들에 대해, 다음 맞춤법 검사기 작동
base_url = 'https://alldic.daum.net/grammar_checker.do'
#웹드라이버 불러오기
driver = webdriver.Chrome('../chromedriver')
driver.get(base_url)

In [23]:
corrected = {}

In [24]:
'''
HTTP request 이용 다음 맞춤법 검사기 실행 코드
코드 수행 중에 pyperclip 패키지를 통해 OS의 복사/붙여넣기 클립보드를 사용하므로,
코드가 돌아가는 중에 다른 곳에서 복사 혹은 붙여넣기를 사용할 경우 데이터에 잘못된 값이 들어갈 수 있으니 주의하시기 바랍니다
'''
for index in train_miss.index :
    text = train_miss.loc[index,'text']
    if len(text) <= 1000:
        driver.get(base_url)
    
        #텍스트 보내기
        driver.find_element_by_id("tfSpelling").send_keys(text)

        #버튼 누르기
        driver.find_element_by_id("btnCheck").click()
    
        try : 
            #복사 버튼 누르기
            driver.find_element_by_id("btnCopy").click()
            corrected[index] = pyperclip.paste()
        except : #교정할 부분이 없을 경우
            corrected[index] = text
            pass
    else:
        text = text[:1000]
        driver.get(base_url)
    
        #텍스트 보내기
        driver.find_element_by_id("tfSpelling").send_keys(text)

        #버튼 누르기
        driver.find_element_by_id("btnCheck").click()
    
        try : 
            #복사 버튼 누르기
            driver.find_element_by_id("btnCopy").click()
            corrected[index] = pyperclip.paste()
        except : #교정할 부분이 없을 경우
            corrected[index] = text
            pass

In [25]:
corrected_txt=pd.Series(corrected)

In [26]:
#맞춤법 검사가 완료된 후 다시 형태소 분석
miss_pos = []
for i, document in enumerate(corrected_txt):
    pos = kom.pos(document)
    clean_words = []
    for p in pos:
        if p[1] == 'NNP' or p[1] == 'NNG' or p[1] == 'SL':
            clean_words.append(p[0])
    document = ' '.join(clean_words)
    miss_pos.append(document)

In [ ]:
train_miss['pos'] = miss_pos

In [ ]:
#한번 교정후 여전히 komoran이 인식 못하는 데이터 확인
train_miss[train_miss['pos']=='']

In [ ]:
#확인 못하는 행을 제외후 나머지만으로 기존 train_miss를 업데이트
train_miss=train_miss[train_miss['pos']!='']

In [ ]:
#train_miss와 train_non_miss를 합친 것으로 train을 업데이트
train = pd.concat([train_miss, train_non_miss], axis=0)

In [ ]:
#정제된 train 파일 내보내기
train.to_csv('train_miss_included_no_userdic.csv', index=False)

## 3. 탐색적 자료분석
## Exploratory Data Analysis


In [5]:
# 스미싱/비 스미싱 각각에 대해 워드 클라우드 생성
# 스미싱/비 스미싱 문자 각각의 길이 히스토그램
# 스미싱/비 스미싱 문자 각각의 형태소 분석 후 단어 개수 히스토그램

## 4. 변수 선택 및 모델 구축
## Feature Engineering & Initial Modeling

In [6]:
#스미싱, 비스미싱으로 나눈 뒤 트레이닝, 벨리데이션 셋 분리
#벨리데이션 셋의 비율은 스미싱/비스미싱 각각 5%
train_sm_list=list(train[train['smishing']==1].index)
val_train_sm_index=random.sample(train_sm_list, 935)

train_nr_list=list(train[train['smishing']!=1].index)
val_train_nr_index=random.sample(train_nr_list, 13861)


train_sm_index = list(set(train_sm_list).difference(set(val_train_sm_index)))
train_nr_index = list(set(train_nr_list).difference(set(val_train_nr_index)))


train_sm = train.loc[train_sm_index,:]
train_nr = train.loc[train_nr_index,:]
training_set = pd.concat([train_sm,train_nr], axis=0)

val_train_sm = train.loc[val_train_sm_index,:]
val_train_nr = train.loc[val_train_nr_index,:]
validation_set = pd.concat([val_train_sm,val_train_nr], axis=0)

X_train =training_set['pos'].values.tolist()
y_train = training_set['smishing'].values.tolist()

X_val =validation_set['pos'].values.tolist()
y_val = validation_set['smishing'].values.tolist()

In [ ]:
#트레이닝 셋과 밸레데이션 셋을 각각 섞어준다. random 시드에 따라 시행 횟수별로 같은 값이 나옴
tmp = [[x,y] for x, y in zip(X_train, y_train)]
random.shuffle(tmp)

tmp1 = [[x,y] for x, y in zip(X_val, y_val)]
random.shuffle(tmp1)

X_train = [n[0] for n in tmp]
y_train = [n[1] for n in tmp]

X_val = [n[0] for n in tmp1]

In [ ]:
X = X_train+X_val
y = y_train+y_val

#토크나이저로 단어들의 사전을 만들고 각 행마다 단어의 인덱스 시퀀스로 만듦
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)

#train 데이터로 학습한 tokenizer를 json파일로 저장
word_vocab = tokenizer.word_index
with open("../wordIndex.json","w") as json_file :
    json_file.write(json.dumps(word_vocab))

X_train_1 = tokenizer.texts_to_sequences(X_train)
X_val_1 = tokenizer.texts_to_sequences(X_val)


max_len=400 # 전체 데이터의 길이는 400으로 맞춘다.
X_train_2 = pad_sequences(X_train_1, maxlen=max_len)
X_val_2 = pad_sequences(X_val_1, maxlen=max_len) #단어 리스트를 (X_train, 400) 크기의 2D 정수 텐서로 변환

word_to_index = tokenizer.word_index
vocab_size = len(word_to_index)+1 

In [ ]:
y_train = np.array(y_train)
y_val = np.array(y_val)

## 5. 모델 학습 및 검증
## Model Tuning & Evaluation

In [7]:
#Simple RNN 모델 
model = Sequential()
model.add(Embedding(vocab_size, 32)) # 임베딩 벡터의 차원은 32
model.add(SimpleRNN(32)) # RNN 셀의 hidden_size는 32
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train_2, y_train, epochs=4, batch_size=32, validation_data=(X_val_2, y_val))

In [ ]:
#모델 저장
model.save('../1_Model/np7_random0_tf8_val0.05.h5')

## 6. 결과 및 결언
## Conclusion & Discussion

In [8]:
# 입력하세요.
